In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from functools import partial
import geopy
import re
import time
from geopy.extra.rate_limiter import RateLimiter
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

In [ ]:
URL = "https://www.travelandleisure.com/attractions/worlds-most-visited-sacred-sites"

In [ ]:
# Header for request
HEADER = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en; q=0.5'})
geolocator = Nominatim(user_agent="geocode_religious_locations_from_webpage")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.5, max_retries=3)
geo_key = 'e0e73007ebe240c6836e278bbc510da2'
geocoder = OpenCageGeocode(geo_key)

In [ ]:
webpage = requests.get(URL, headers=HEADER)
webpage

In [ ]:
soup = BeautifulSoup(webpage.content, "html.parser")
#soup

In [ ]:
titles = soup.find_all("span", attrs = {'class': 'mntl-sc-block-heading__text'})
data = soup.find_all("p", attrs = {'class': 'comp mntl-sc-block mntl-sc-block-html'})
#print(data[53].text.strip("\n").split(":")[1].strip().split(" ")[0].split("–")[0])
#len(data)
temp_title = titles[1].text.strip()
title = re.sub(r"No\. [0-9]\s", "", temp_title)
title

In [ ]:
for i in range(20):
    temp_title = titles[i].text.strip()
    title = re.sub(r"No\. \b([1-9]|[12][0-9])\b\s", "", temp_title)
    #print(str(i) + "     " + title)
    titles[i] = title


In [ ]:
visitors_list = []
sample_word = "Annual Visitors:"
for i in range(len(data)):
    if sample_word in data[i].text:
        #print(i)
        visitor = data[i].text.strip("\n").split(":")[1].strip().split(" ")[0].split("–")[0]
        #print(str(i) + "  " + visitor)
        visitors_list.append(visitor)
        if len(visitors_list) == 20:
            break
visitors_list

In [ ]:
final_title_list = titles[:21] + visitors_list
#for i in range(len(final_title_list)):
    #print(str(i)+ "  " + final_title_list[i])
print(len(final_title_list))
print(final_title_list[0])
print(final_title_list[1])

In [ ]:
city_address = []
for i in range(int(len(final_title_list)/2)):

    # GETTING RAW DATA
    try:
        location_description = geocoder.geocode(final_title_list[i])
    except Exception:
        print(str(i) + " " + "No data found for location: " + final_title_list[i])
        continue

    # GETTING ADDRESS VALUES
    loc_title = final_title_list[i].split(",")[0]
    latitude = str(location_description[0]['geometry']['lat'])
    longitude = str(location_description[0]['geometry']['lng'])
    try:
        city = location_description[0]['components']['city']
    except Exception:
        try:
            city = location_description[0]['components']['town']
        except Exception:
            try:
                city = location_description[0]['components']['state']
            except Exception:
                city = "NULL"
    country = location_description[0]['components']['country']
    country_code = location_description[0]['components']['country_code']
    continent = location_description[0]['components']['continent']
    visitors = final_title_list[i+21]
    
    #GETTING POSTCODE
    lat_long = str(latitude) + ","+ str(longitude) 
    location2 = geolocator.reverse(lat_long, language="en")
    try:
        postcode = location2.raw['address']['postcode'].replace(" ","")
    except:
        postcode = "NULL"
        
    # ADDING VARIABLES TO DICTIONARY
    address_dict = {'Location': loc_title, 'City/State': city, 'Country': country, 'Country Code':country_code.upper(), 'Continent': continent, 'Latitude': str(round(float(latitude),2)), 'Longitude': str(round(float(longitude),2)), 'Postcode': postcode, 'Visitors (in Millions)': visitors}
    city_address.append(address_dict)
    
print(city_address)

In [ ]:
ca_most_religious_df = pd.DataFrame.from_dict(city_address)
ca_most_religious_df